In [2]:
from __future__ import print_function
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [3]:
# loading data and maximum length of a sequence
max_features = 20000
maxlen = 30
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


/Users/danielcoelho/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/keras/datasets/imdb.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/danielcoelho/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/keras/datasets/imdb.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
 #Here's what an example sequence looks like
x_train[123,:]

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                       kernel_initializer = initializers.RandomNormal(stddev = 0.001),
                       recurrent_initializer = initializers.Identity(gain = 1.0),
                       activation = 'relu',
                       input_shape = x_train.shape[1:]))
model_rnn.add(Dense(1, activation = 'sigmoid'))

2021-11-08 16:02:29.610973: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-08 16:02:29.611210: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5)                 280       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(lr = 0.0001)

model_rnn.compile(loss = 'binary_crossentropy',
                 optimizer = rmsprop,
                 metrics = ['accuracy'])

In [10]:
model_rnn.fit(x_train, y_train,
             batch_size = batch_size,
             epochs = 10,
             validation_data = (x_test, y_test))

/Users/danielcoelho/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6792 - accuracy: 0.5813 - val_loss: 0.6462 - val_accuracy: 0.6756
Epoch 2/10
25000/25000 [==============================] - 28s 1ms/step - loss: 0.6034 - accuracy: 0.6968 - val_loss: 0.5852 - val_accuracy: 0.6986
Epoch 3/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.5372 - accuracy: 0.7365 - val_loss: 0.5428 - val_accuracy: 0.7248
Epoch 4/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.4949 - accuracy: 0.7635 - val_loss: 0.5233 - val_accuracy: 0.7360
Epoch 5/10
25000/25000 [==============================] - 28s 1ms/step - loss: 0.4651 - accuracy: 0.7824 - val_loss: 0.5045 - val_accuracy: 0.7468
Epoch 6/10
25000/25000 [==============================] - 27s 1ms/step - loss: 0.4413 - accuracy: 0.7952 - val_loss: 0.4901 - val_accuracy: 0.7574
Epoch 7/10
25000/25000 [==============================] - 27s 1ms/st

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                               batch_size = batch_size)
print('Test score:', score)
print('Test accuracy', acc)

25000/25000 [==============================] - 3s 138us/step
Test score: 0.4571345684337616
Test accuracy 0.7816399931907654


## Exercise
### Your Turn

Now do it yourself:
- Prepare the data to use sequences of length 80 rather than length 30.  Did it improve the performance?
- Try different values of the "max_features".  Can you improve the performance?
- Try smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [12]:
max_features = 20000
maxlen = 80

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)

x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)

/Users/danielcoelho/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/keras/datasets/imdb.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/danielcoelho/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/keras/datasets/imdb.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [13]:
# Out of curiosity, run for 10 more epochs
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                       kernel_initializer = initializers.RandomNormal(stddev = 0.001),
                       recurrent_initializer = initializers.Identity(gain = 1.0),
                       activation = 'relu',
                       input_shape = x_train.shape[1:]))
model_rnn.add(Dense(1, activation = 'sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(lr = 0.0001)

model_rnn.compile(loss = 'binary_crossentropy',
                 optimizer = rmsprop,
                 metrics = ['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
             batch_size = batch_size,
             epochs = 10,
             validation_data = (x_test, y_test))

/Users/danielcoelho/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 50s 2ms/step - loss: 0.6027 - accuracy: 0.6715 - val_loss: 0.5292 - val_accuracy: 0.7312
Epoch 2/10
25000/25000 [==============================] - 48s 2ms/step - loss: 0.4601 - accuracy: 0.7841 - val_loss: 0.4666 - val_accuracy: 0.7767
Epoch 3/10
25000/25000 [==============================] - 48s 2ms/step - loss: 0.3944 - accuracy: 0.8249 - val_loss: 0.4483 - val_accuracy: 0.7820
Epoch 4/10
25000/25000 [==============================] - 49s 2ms/step - loss: 0.3495 - accuracy: 0.8495 - val_loss: 0.4019 - val_accuracy: 0.8149
Epoch 5/10
25000/25000 [==============================] - 49s 2ms/step - loss: 0.3192 - accuracy: 0.8644 - val_loss: 0.3846 - val_accuracy: 0.8276
Epoch 6/10
25000/25000 [==============================] - 49s 2ms/step - loss: 0.2991 - accuracy: 0.8736 - val_loss: 0.3773 - val_accuracy: 0.8318
Epoch 7/10
25000/25000 [==============================] - 49s 2ms/st

In [16]:
max_features = 500
maxlen = 80

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)

x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)

/Users/danielcoelho/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/keras/datasets/imdb.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/danielcoelho/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/keras/datasets/imdb.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                       kernel_initializer = initializers.RandomNormal(stddev = 0.001),
                       recurrent_initializer = initializers.Identity(gain = 1.0),
                       activation = 'relu',
                       input_shape = x_train.shape[1:]))
model_rnn.add(Dense(1, activation = 'sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(lr = 0.0001)

model_rnn.compile(loss = 'binary_crossentropy',
                 optimizer = rmsprop,
                 metrics = ['accuracy'])

In [19]:
model_rnn.fit(x_train, y_train,
             batch_size = batch_size,
             epochs = 10,
             validation_data = (x_test, y_test))

/Users/danielcoelho/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 39s 2ms/step - loss: 0.6416 - accuracy: 0.6216 - val_loss: 0.5732 - val_accuracy: 0.7015
Epoch 2/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.5475 - accuracy: 0.7214 - val_loss: 0.5513 - val_accuracy: 0.7186
Epoch 3/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.5150 - accuracy: 0.7432 - val_loss: 0.5087 - val_accuracy: 0.7458
Epoch 4/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4933 - accuracy: 0.7590 - val_loss: 0.4877 - val_accuracy: 0.7581
Epoch 5/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4768 - accuracy: 0.7715 - val_loss: 0.4793 - val_accuracy: 0.7669
Epoch 6/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4655 - accuracy: 0.7793 - val_loss: 0.4650 - val_accuracy: 0.7757
Epoch 7/10
25000/25000 [==============================] - 37s 1ms/st

In [20]:
model_rnn.fit(x_train, y_train,
             batch_size = batch_size,
             epochs = 10,
             validation_data = (x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4433 - accuracy: 0.7897 - val_loss: 0.4486 - val_accuracy: 0.7870
Epoch 2/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4417 - accuracy: 0.7913 - val_loss: 0.4460 - val_accuracy: 0.7888
Epoch 3/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4403 - accuracy: 0.7926 - val_loss: 0.4444 - val_accuracy: 0.7907
Epoch 4/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4390 - accuracy: 0.7937 - val_loss: 0.4438 - val_accuracy: 0.7908
Epoch 5/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4381 - accuracy: 0.7950 - val_loss: 0.4636 - val_accuracy: 0.7780
Epoch 6/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4371 - accuracy: 0.7944 - val_loss: 0.4462 - val_accuracy: 0.7894
Epoch 7/10
25000/25000 [==============================] - 37s 1ms/st